In [2]:
from dotenv import load_dotenv

import openai
import os
load_dotenv()

import logging
import sys
import os

import numexpr as ne

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
openai.api_key = os.getenv('OPENAI_API_KEY')


os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'

In [3]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from llama_index.prompts import PromptTemplate

try:
    storage_context = StorageContext.from_defaults(persist_dir='../storage/cache/andrew/medical')
    index = load_index_from_storage(storage_context)
    print('loading from disk')
except:
    documents = SimpleDirectoryReader('../assets/AndrewHuberman/medical').load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir='../storage/cache/andrew/medical/')
    print('persisting to disk')

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
loading from disk


In [4]:
openai.log = "debug"

text_qa_template_str = (
    "Bạn là trợ lý của Andrew Huberman, người có thể đọc các ghi chú podcast của Andrew Huberman.\n"
    "Luôn trả lời truy vấn chỉ bằng cách sử dụng thông tin ngữ cảnh được cung cấp, "
    "chứ không phải kiến ​​thức sẵn có.\n"
    "Một số quy tắc cần tuân theo:\n"
    "1. Không bao giờ tham khảo trực tiếp bối cảnh nhất định trong câu trả lời của bạn.\n"
    "2. Tránh những câu như 'Dựa trên ngữ cảnh, ...' hoặc "
    "'Thông tin ngữ cảnh ...' hoặc bất cứ điều gì cùng"
    "những dòng đó."
    "3.Câu trả lời nên thật chi tiết và dài"
    "Thông tin bối cảnh dưới đây.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Trả lời câu hỏi:{query_str}\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [5]:
from llama_index.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    model="sentence-transformers/all-mpnet-base-v2", top_n=3
)

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from time import time

query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[rerank],
    text_qa_template = text_qa_template
)

In [7]:
now = time()
response = query_engine.query(
    "Bệnh sán dây là bệnh gì?",
)
print(f"Elapsed: {round(time() - now, 2)}s")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Elapsed: 12.16s


In [8]:
print(response)

Bệnh sán dây là một loại nhiễm trùng do trứng sán dây (sán dây lợn, sán dây bò, sán dây cá...) xâm nhập vào cơ thể người thông qua đường miệng, đặc biệt là do thói quen ăn thịt sống, thịt tái hoặc chưa nấu chín. Cơ thể bị nhiễm sán dây có thể gặp các triệu chứng nhẹ và không đặc hiệu như mệt mỏi, đau bụng, tiêu chảy. Các triệu chứng cũng có thể khác nhau tùy thuộc vào từng loại sán dây mắc phải.


In [9]:
print(response.get_formatted_sources(length=200))

> Source (Doc id: 4b03076e-9f80-4a81-92db-d782edd06c26): Nguyên nhân do trứng sán dây (sán dây lợn, sán dây bò, sán dây cá…) xâm nhập từ động vật vào người qua đường miệng, đặc biệt là thói quen ăn thịt sống, thịt tái hoặc chưa nấu chín. Thông thường, cơ...

> Source (Doc id: ac765d72-fc26-4784-a65f-f8ed25f3eaa8): Loại nhiễm trùng này xảy ra với sán dây lùn và phổ biến hơn nhiều ở những nơi thực hành vệ sinh kém. 
<h3>Tái nhiễm sán</h3> Trong quá trình điều trị, con người có thể tự tái nhiễm nếu không tuân t...

> Source (Doc id: 12d19046-ed27-48ec-a546-5305674ac6c7): Ngoài ra, hãy đến bệnh viện dù cho bạn không gặp bất kỳ triệu chứng nào của bệnh. Kiểm tra sức khỏe định kỳ cũng là một cách phát hiện bệnh từ sớm và điều trị kịp thời. 
<h2><strong>9. Cách phòng t...


In [10]:
print(response.metadata)

{'4b03076e-9f80-4a81-92db-d782edd06c26': {'file_path': '../assets/AndrewHuberman/medical/benh-san-day', 'file_name': 'benh-san-day', 'file_type': None, 'file_size': 19844, 'creation_date': '2023-12-15', 'last_modified_date': '2023-12-15', 'last_accessed_date': '2023-12-15'}, 'ac765d72-fc26-4784-a65f-f8ed25f3eaa8': {'file_path': '../assets/AndrewHuberman/medical/benh-san-day', 'file_name': 'benh-san-day', 'file_type': None, 'file_size': 19844, 'creation_date': '2023-12-15', 'last_modified_date': '2023-12-15', 'last_accessed_date': '2023-12-15'}, '12d19046-ed27-48ec-a546-5305674ac6c7': {'file_path': '../assets/AndrewHuberman/medical/liet-day-thanh-quan', 'file_name': 'liet-day-thanh-quan', 'file_type': None, 'file_size': 18974, 'creation_date': '2023-12-15', 'last_modified_date': '2023-12-15', 'last_accessed_date': '2023-12-15'}}


In [11]:
response.response

'Bệnh sán dây là một loại nhiễm trùng do trứng sán dây (sán dây lợn, sán dây bò, sán dây cá...) xâm nhập vào cơ thể người thông qua đường miệng, đặc biệt là do thói quen ăn thịt sống, thịt tái hoặc chưa nấu chín. Cơ thể bị nhiễm sán dây có thể gặp các triệu chứng nhẹ và không đặc hiệu như mệt mỏi, đau bụng, tiêu chảy. Các triệu chứng cũng có thể khác nhau tùy thuộc vào từng loại sán dây mắc phải.'

In [14]:
# Embedding Similarity Evaluator

from llama_index.evaluation import SemanticSimilarityEvaluator
from llama_index import ServiceContext
from llama_index.embeddings import SimilarityMode


# service_context = ServiceContext.from_defaults(embed_model="local")
evaluator = SemanticSimilarityEvaluator(
    # service_context=service_context,
    similarity_mode=SimilarityMode.DEFAULT,
    similarity_threshold=0.9,
)


reference = """
<h2><strong>Xuất tinh sớm là gì?</strong></h2> <strong>Xuất tinh sớm</strong> (premature ejaculation) là tình trạng rối loạn xuất tinh khi người đàn ông không thể làm chủ được thời gian xuất tinh, quá trình xuất tinh sớm được tính từ khi dương vật đưa vào âm đạo chỉ duy trì trong vòng 2 phút. Quá trình xuất tinh là sự phóng tinh dịch ra khỏi dương vật khi đạt được cực khoái. Việc xuất tinh sớm không chỉ ảnh hưởng đến của cảm xúc giường chiếu của chính mình mà “đối tác” của họ cũng ảnh hưởng không kém. Xuất tinh sớm là hiện tượng khá phổ biến, xuất hiện từ độ tuổi 18 đến 59. (1) Xuất tinh sớm là vấn đề này khá nhạy cảm nên nhiều người không dám đối diện, né tránh thăm khám, chữa trị. Tuy nhiên, điều này ảnh hưởng rất lớn đến cuộc sống của nam giới, nếu không được điều trị sớm, nam giới có nguy cơ đối mặt với những biến chứng nguy hiểm như: rối loạn cương dương, gây liệt dương, thậm chí dẫn đến vô sinh. Xuất tinh sớm thường có 4 dạng là nguyên phát (thời gian xuất tinh ngắn xảy ra ngay từ những lần đầu giao hợp và kéo dài cho tới hiện tại), thứ phát (thời gian xuất tinh bình thường, nhưng thời gian gây đây có giảm đáng kể ảnh và ảnh hưởng tới tâm lý và quan hệ cặp đôi), xuất tinh sớm thay đổi (thời gian xuất tinh thay đổi tùy thuộc vào hoàn cảnh, đối tác, kích thích…) và các rối loạn xuất tinh giống <strong>xuất tinh sớm</strong>.
"""


result = await evaluator.aevaluate(
    response=response.response,
    reference=reference,
)

print("Score: ", result.score)
print("Passing: ", result.passing)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Score:  0.8039874095294477
Passing:  False
